## Importação dos módulos necessários

In [29]:
# importando conexão com banco de dados pyodbc
import pyodbc

# importando pandas
import pandas as pd

# alterando número de linhas
pd.set_option('display.max_rows', 200)

# alterando número de colunas
pd.set_option('display.max_columns', 70)

#  Módulo para leitura de arquivos em excel
import openpyxl

# modulo pára leitura de arquivos excel
import xlrd


## Diretório dos arquivos

In [91]:
# Diretorio de base
diretorio_base = 'G:/Meu Drive/TARUMA/PRONTUARIO_CONDUTOR/'

# Bancos de dados
banco_de_dados = diretorio_base + 'SISTEMA_DE_GERENCIAMENTO_DE_CONDUTORES.accdb'

# relatorios de importacao 1: ranking do motorista 2: dados_motoristas
# Relatorio de importacao
relatorio_importacao = diretorio_base + '/RELATORIOS/RANK_MOTORISTA.xlsx'

# Dados dos motoristas
dados_motoristas = diretorio_base + 'RELATORIOS/MOT_.xls'

In [131]:
# execução de queries no banco de dados
def executa_sql(sql: str):

    # inicio a conexão
    connection = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + banco_de_dados + ';')

    #  Gero o curso
    cursor = connection.cursor()

    #  executo a query
    cursor.execute(sql)

    #  faço o commit da transação

    connection.commit()

    # fecho o cursor
    cursor.close()

    # fecho ao conexao
    connection.close()


def conexao_banco_de_dados():

    # inicio a conexão
    return  pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + banco_de_dados + ';')


In [19]:
# lista de caracteres para serem alterados
special_characters = {'á': 'a',
                       'à': 'a',
                       'é': 'e',
                       'è': 'e',
                       'í': 'i',
                       'ì': 'i',
                       'ó': 'o',
                       'ò': 'o',
                       'ú': 'u',
                       'ù': 'u'}


def swap_special_characters(column):
    for char in column:
        if char in special_characters:
            column = column.replace(char, special_characters[char])
    return column


def normalize_columns(columns):

    return columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.replace(' ', '_')


In [20]:
# importando arquivo excel
ranking_condutor = pd.read_excel(relatorio_importacao, engine='openpyxl', skiprows=1)

# normalizando todos caracteres dos nomes das colunas
ranking_condutor.columns = normalize_columns(ranking_condutor.columns)

# Removendo caracteres inválidos
ranking_condutor.columns = ranking_condutor.columns.str.replace('[^A-Za-z0-9]+', '_', regex=True )

# convertendo data para datetime
ranking_condutor.data = pd.to_datetime(ranking_condutor.data)

# Dados dos motoristas
motoristas = pd.read_excel(dados_motoristas, engine='xlrd')

# Normalizando os nomes nas colunas
motoristas.columns = normalize_columns(motoristas.columns)

# removendo caracteres no cpf
motoristas['cpf'] = motoristas.cpf.str.replace('.', '', regex=False).str.replace('-', '', regex=False)

# removendo dados inválidos nos nomes dos motoristas
motoristas.dropna(subset=['codigo_do_motorista'], inplace=True)

# colocando código do motorista como número inteiro
motoristas.codigo_do_motorista = motoristas.codigo_do_motorista.astype(int)

# Limpando  os dados inválidos para os códigos dos motoristas
ranking_condutor.dropna(subset=['codigo'], inplace=True)

# Convertendo para inteiro
ranking_condutor.codigo = ranking_condutor.codigo.astype(int)

## gerando dataset com o merge dos dados
dados = motoristas[['cpf', 'codigo_do_motorista']].merge(
    ranking_condutor,
    left_on='codigo_do_motorista',
    right_on='codigo',
    how= 'inner'
)


In [137]:
total_itens = dados.shape[0]
erros = []

for index, row in dados.iterrows():

    try:
        sql = f"""
        INSERT INTO TB_TELEMETRIA(
            COD_MOTORISTA,
            CPF_FUNCIONARIO,
            DEPARTAMENTO_MOTORISTA,
            DATA,
            CURVA_BRUSCA_QUANTIDADE,
            ACELERACAO_QUANTIDADE,
            FRENAGEM_QUANTIDADE,
            EXCESSO_VELOCIDADE_QUANTIDADE,
            EXCESSO_VELOCIDADE_OCORRENCIA,
            EXCESSO_VELOCIDADE_NAO_TOLERADO
        )
        values(
            {row.codigo},
            '{row.cpf}',
            '{row.departamento}',
            '{row.data}',
            {row.iloc[14]},
            {row.iloc[16]},
            {row.iloc[18]},
            {row.iloc[21]},
            {row.iloc[23]},
            {row.iloc[24]}
        )
        """
        executa_sql(sql)

    except:
        erros.append([ index, row.cpf, row.nome])


## Adicionando nomes que estão faltando no banco de dados

In [127]:
dados_erros = pd.DataFrame(erros)
dados_erros.head()

,0,1,2
0,58,07642268602,ANDRE LUIS DE JESUS BARRA
1,59,07642268602,ANDRE LUIS DE JESUS BARRA
2,60,07642268602,ANDRE LUIS DE JESUS BARRA
3,61,07642268602,ANDRE LUIS DE JESUS BARRA
4,62,07642268602,ANDRE LUIS DE JESUS BARRA
5,63,07642268602,ANDRE LUIS DE JESUS BARRA
6,64,07642268602,ANDRE LUIS DE JESUS BARRA
7,65,07642268602,ANDRE LUIS DE JESUS BARRA
8,66,07642268602,ANDRE LUIS DE JESUS BARRA
9,67,07642268602,ANDRE LUIS DE JESUS BARRA


In [117]:
for index,row in  dados_erros[[1,2]].drop_duplicates().iterrows():
    try:
        sql = f""" INSERT INTO TB_FUNCIONARIO (CPF_FUNCIONARIO, DESC_FUNCIONARIO) VALUES('{row[1]}', '{row[2]}')"""
        executa_sql(sql)
    except:
        print('Não foi possível registrar os dados')

Não foi possível registrar os dados
Não foi possível registrar os dados
Não foi possível registrar os dados
Não foi possível registrar os dados
Não foi possível registrar os dados
Não foi possível registrar os dados
Não foi possível registrar os dados
Não foi possível registrar os dados
Não foi possível registrar os dados


## Atualização de registros e trabelas

In [148]:
sql = "UPDATE TB_TELEMETRIA SET CURVA_BRUSCA_QUANTIDADE=0,EXCESSO_VELOCIDADE_QUANTIDADE=0, EXCESSO_VELOCIDADE_OCORRENCIA=0, EXCESSO_VELOCIDADE_NAO_TOLERADO=0"
executa_sql(sql)

In [149]:
prontuario = pd.read_sql(sql="select * from qry_prontuario_condutor_classificado'", con=conexao_banco_de_dados()).fillna(0)

C:\Users\Anderson\AppData\Local\Temp\ipykernel_21252\2235218846.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prontuario = pd.read_sql(sql="select * from qry_prontuario_condutor_classificado'", con=conexao_banco_de_dados()).fillna(0)


In [150]:
prontuario

,NOME,DEPTO,FREIO,ACELERA,CURVA,VELOCIDADE0,VELOCIDADE1,VELOCIDADE2,MULTAS,ADVERTENCIA,AVARIAS,SOMA_PONTOS
0,ANTONIO AUGUSTO DA SILVA,ROTA BARBACENA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0,3.0
1,ELIESIO ALVES DO NASCIMENTO,ROTA BARBACENA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,JOSE ANGELO DA FONSECA,ROTA BARBACENA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0,3.0
3,JOSE ANISIO DA SILVA,ROTA BARBACENA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,JOSE CARLOS ALMEIDA,ROTA BARBACENA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
5,MARCO ANTONIO DIAS DA SILVA,ROTA BARBACENA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
6,ROGERIO ALVES DO NASCIMENTO,ROTA BARBACENA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
7,WESLLEY MATEUS DA SILVA,ROTA BARBACENA,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
8,AILTON ALVES DE SIQUEIRA,ROTA JUIZ DE FORA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0,3.0
9,ALEX ANTONIO DE SOUZA,ROTA JUIZ DE FORA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
